In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import sys

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

import time

DATASET_PATH = Path("./data/text")

VOC_SIZE = 1000

def load_data(datapath, max_size=None):
    texts_files = list(datapath.glob("*.txt"))
    texts = []  
    for files in texts_files:
        with open(files, "r", encoding='utf8') as files:
            text = files.readlines()
            texts += text
    texts = list(set(texts))
    
    return texts

texts = load_data(DATASET_PATH)

from tokenizers import Tokenizer
# from transformers import AutoTokenizer
from transformers import BertTokenizer, BertForMaskedLM
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
from tokenizers.pre_tokenizers import Whitespace


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model_checkpoint = "distilgpt2"
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenizer.pre_tokenizer = Whitespace()
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

inputs = tokenizer(texts, return_tensors='pt', max_length=100
                   , truncation=True, padding='max_length')

inputs['labels'] = inputs.input_ids.detach().clone()

rand = torch.rand(inputs.input_ids.shape)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [2]:
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [3]:
import random as rd

In [4]:
selection = []

for i in range(mask_arr.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
        )

selection[:5]

for i in range(mask_arr.shape[0]):
    inputs.input_ids[i, selection[i]] = 103 # application du token [MASK]

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, idx):
        self.encodings = encodings
        self.idx = idx
        self.encodings = {key: [val[i] for i in self.idx] for key, val in self.encodings.items()}
        
    def __getitem__(self, idx):
        return {key : torch.tensor(val[idx]) for key, val in self.encodings.items()}
    
    def __len__(self):
        return len(self.encodings['input_ids'])

sample_idx = [i for i in range(len(inputs.input_ids))]

shuffled_sample_idx = rd.sample(sample_idx, len(sample_idx))

train_idx = shuffled_sample_idx[:int(0.70*len(shuffled_sample_idx))]
val_idx = shuffled_sample_idx[int(0.70*len(shuffled_sample_idx)):int(0.85*len(shuffled_sample_idx))]
test_idx = shuffled_sample_idx[int(0.85*len(shuffled_sample_idx)):]
                                
dataset_train = CustomDataset(inputs, train_idx)
dataset_val = CustomDataset(inputs, val_idx)
dataset_test = CustomDataset(inputs, test_idx)

train_dataloaded = torch.utils.data.DataLoader(dataset_train, batch_size=16, shuffle=True)
val_dataloaded = torch.utils.data.DataLoader(dataset_val, batch_size=16, shuffle=True)
test_dataloaded = torch.utils.data.DataLoader(dataset_test, batch_size=16, shuffle=True)

In [5]:
#class MLM_model(nn.Module):
#    def __init__(self, model):
#        super(MLM_model, self).__init__()
#        self.history = {"epochs":[], "test":[]}
#        self.model = model
    
#    def parameters(self):
#        return self.model.parameters()

#    def forward(self, x, attention_mask, labels):
#        return self.model(x, attention_mask, labels)
    
#    def train_log(self, train_batch_losses, val_batch_losses, train_loss, validation_loss):
#        self.history["epochs"].append({"train_batch_losses":train_batch_losses, 
#                                "val_batch_losses":val_batch_losses, 
#                                "train_loss":train_loss, 
#                                "validation_loss":validation_loss})
    
#    def test_log(self, test_batch_losses, test_loss):
#        self.history["test"].append({"test_batch_losses":test_batch_losses,
#                                "test_loss":test_loss})

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#model = MLM_model(model)
model.to(device)
print(device)

cuda


In [7]:
def train_step(module, batch, batch_idx, optimizer):
    module.train(True)
    
    inputs_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    
    outputs = module(inputs_ids, attention_mask, labels=labels)
    
    loss = outputs.loss
    print(f"\n\033[1;37mBatch loss {batch_idx+1} : {loss.item()}")
    loss.backward()
    
    torch.nn.utils.clip_grad_norm_(module.parameters(), max_norm=1.0)
    optimizer.step()
    optimizer.zero_grad()
    
    return module, loss

def eval_step(module, batch, batch_idx, optimizer=None, training=True):
    with torch.no_grad():
        
        inputs_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
    
        outputs = module(inputs_ids, attention_mask, labels=labels)
    
        loss = outputs.loss
         
        if training:
            print(f"\n\033[1;32mValidation Batch loss {batch_idx+1} : {loss.item()}")
            return module, loss
        else:
            print(f"\n\033[1;32mTest Batch loss {batch_idx+1} : {loss.item()}")
            return module, loss, outputs, labels

def train_loop(module, EPOCHS, train_dataset, val_dataset, optimizer, lr_scheduler=None):
    for epoch in range(EPOCHS):
        deb=time.time()
        
        module.train(True)
        
        train_batch_losses = []
        for batch_idx in range(len(train_dataset)):
            batch = next(iter(train_dataset))
            module, loss = train_step(module, batch, batch_idx, optimizer)
            train_batch_losses.append(loss.item())
            
        if lr_scheduler is not None:
          lr_scheduler.step()
        train_loss = np.mean(train_batch_losses)

        module.train(False)
        val_batch_losses = []
        for batch_idx in range(len(val_dataset)):
            batch = next(iter(val_dataset))
            module, loss = eval_step(module, batch, batch_idx)
            val_batch_losses.append(loss.item())
        val_loss = np.mean(val_batch_losses)

#        module.train_log(train_batch_losses, val_batch_losses, train_loss, val_loss)
        print(f"\n\033[1;33mEpoch {epoch+1} :\n\033[1;37mTraining Loss : {train_loss}")
        print(f"\033[1;32mValidation Loss : {val_loss}")
        print(f"\033[1;31mDurée epoch : {time.time()-deb} secondes")
    return module

def evaluate(module, test_dataset):
    module.train(False)
    test_batch_losses = []
    predictions = []
    true_targets = []
    for batch_idx in range(len(test_dataset)):
        batch = next(iter(test_dataset))
        module, loss, outputs, labels = eval_step(module, batch, batch_idx, training=False)

        test_batch_losses.append(loss.item())
        predictions.append(outputs)
        true_targets.append(labels)

    test_loss = np.mean(test_batch_losses)
#    module.test_log(test_batch_losses, test_loss)
    print(f"\nTest Loss : {test_loss}")
    return predictions, true_targets

In [8]:
if __name__ == "__main__":
    EPOCHS = 1
    LR = 1e-4
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LR, eps=5e-8)
    module = train_loop(module=model,
                        EPOCHS=EPOCHS, 
                        train_dataset=train_dataloaded, 
                        val_dataset=val_dataloaded,
                        optimizer=optimizer)
    predictions, true_targets = evaluate(module, 
                                         test_dataloaded)



C:\Users\Mateo\AppData\Local\Temp\ipykernel_15904\3715340537.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key : torch.tensor(val[idx]) for key, val in self.encodings.items()}



Batch loss 1 : 11.209537506103516

Batch loss 2 : 7.477960109710693

Batch loss 3 : 5.865095615386963

Batch loss 4 : 5.1089959144592285

Batch loss 5 : 4.225607395172119

Batch loss 6 : 3.7812817096710205

Batch loss 7 : 3.0029916763305664

Batch loss 8 : 2.6055808067321777

Batch loss 9 : 2.048874855041504

Batch loss 10 : 1.6938650608062744

Batch loss 11 : 1.3665978908538818

Batch loss 12 : 1.118104100227356

Batch loss 13 : 0.947327196598053

Batch loss 14 : 0.797278881072998

Batch loss 15 : 0.6209564805030823

Batch loss 16 : 0.47918128967285156

Batch loss 17 : 0.4530010223388672

Batch loss 18 : 0.38324710726737976

Batch loss 19 : 0.3482261598110199

Batch loss 20 : 0.39186739921569824

Batch loss 21 : 0.3628880977630615

Batch loss 22 : 0.16519016027450562

Batch loss 23 : 0.2534430921077728

Batch loss 24 : 0.1652783751487732

Batch loss 25 : 0.10932251811027527

Batch loss 26 : 0.16025595366954803

Batch loss 27 : 0.16873598098754883

Batch loss 28 : 0.15883861482143402



Batch loss 187 : 0.12530499696731567

Batch loss 188 : 0.15366123616695404

Batch loss 189 : 0.0313909687101841

Batch loss 190 : 0.07675524801015854

Batch loss 191 : 0.050781283527612686

Batch loss 192 : 0.03863641992211342

Batch loss 193 : 0.014467110857367516

Batch loss 194 : 0.10684517025947571

Batch loss 195 : 0.022875027731060982

Batch loss 196 : 0.06312664598226547

Batch loss 197 : 0.07000399380922318

Batch loss 198 : 0.050838932394981384

Batch loss 199 : 0.03009534440934658

Batch loss 200 : 0.06679247319698334

Batch loss 201 : 0.09468068927526474

Batch loss 202 : 0.08365478366613388

Batch loss 203 : 0.082374706864357

Batch loss 204 : 0.10345688462257385

Batch loss 205 : 0.0798397958278656

Batch loss 206 : 0.06753335148096085

Batch loss 207 : 0.08342394977807999

Batch loss 208 : 0.04932736977934837

Batch loss 209 : 0.13346032798290253

Batch loss 210 : 0.12627698481082916

Batch loss 211 : 0.05256440117955208

Batch loss 212 : 0.08994582295417786

Batch loss 


Batch loss 369 : 0.05187772214412689

Batch loss 370 : 0.05368535965681076

Batch loss 371 : 0.06365494430065155

Batch loss 372 : 0.07361477613449097

Batch loss 373 : 0.07380446791648865

Batch loss 374 : 0.03696180880069733

Batch loss 375 : 0.06576213985681534

Batch loss 376 : 0.08993874490261078

Batch loss 377 : 0.03941306471824646

Batch loss 378 : 0.0406225211918354

Batch loss 379 : 0.03402959927916527

Batch loss 380 : 0.03676323592662811

Batch loss 381 : 0.04405699670314789

Batch loss 382 : 0.03346351161599159

Batch loss 383 : 0.029890011996030807

Batch loss 384 : 0.04372113570570946

Batch loss 385 : 0.030780578032135963

Batch loss 386 : 0.07434301823377609

Batch loss 387 : 0.034444235265254974

Batch loss 388 : 0.05857133865356445

Batch loss 389 : 0.07760950922966003

Batch loss 390 : 0.039407290518283844

Batch loss 391 : 0.04450587183237076

Batch loss 392 : 0.0327792763710022

Batch loss 393 : 0.038002971559762955

Batch loss 394 : 0.009760660119354725

Batch l


Batch loss 550 : 0.021675825119018555

Batch loss 551 : 0.066909559071064

Batch loss 552 : 0.062127407640218735

Batch loss 553 : 0.028630787506699562

Batch loss 554 : 0.01588056981563568

Batch loss 555 : 0.030163416638970375

Batch loss 556 : 0.02417643368244171

Batch loss 557 : 0.01817144826054573

Batch loss 558 : 0.08265826851129532

Batch loss 559 : 0.04748556762933731

Batch loss 560 : 0.05598694831132889

Batch loss 561 : 0.025976872071623802

Batch loss 562 : 0.01728919893503189

Batch loss 563 : 0.05843299254775047

Batch loss 564 : 0.02100163884460926

Batch loss 565 : 0.024281740188598633

Batch loss 566 : 0.03241710364818573

Batch loss 567 : 0.03737618774175644

Batch loss 568 : 0.04886927083134651

Batch loss 569 : 0.03541106358170509

Batch loss 570 : 0.04875022545456886

Batch loss 571 : 0.025541499257087708

Batch loss 572 : 0.08998016268014908

Batch loss 573 : 0.015683196485042572

Batch loss 574 : 0.024593288078904152

Batch loss 575 : 0.04550711810588837

Batc

...


Batch loss 306 : 0.024215171113610268

Batch loss 307 : 0.017945365980267525

Batch loss 308 : 0.02387925423681736

Batch loss 309 : 0.0337100587785244

Batch loss 310 : 0.03290671855211258

Batch loss 311 : 0.03529256209731102

Batch loss 312 : 0.02170977182686329

Batch loss 313 : 0.002133465139195323

Batch loss 314 : 0.024184202775359154

Batch loss 315 : 0.022522881627082825

Batch loss 316 : 0.011701328679919243

Batch loss 317 : 0.02926434576511383

Batch loss 318 : 0.05143081769347191

Batch loss 319 : 0.01325247809290886

Batch loss 320 : 0.013901717960834503

Batch loss 321 : 0.01847996935248375

Batch loss 322 : 0.01997978240251541

Batch loss 323 : 0.014888517558574677

Batch loss 324 : 0.017300819978117943

Batch loss 325 : 0.06813787668943405

Batch loss 326 : 0.03943759575486183

Batch loss 327 : 0.01522519439458847

Batch loss 328 : 0.018011827021837234

Batch loss 329 : 0.03289113566279411

Batch loss 330 : 0.02685389667749405

Batch loss 331 : 0.03739719092845917

Ba


Batch loss 487 : 0.013446156866848469

Batch loss 488 : 0.017804134637117386

Batch loss 489 : 0.026222912594676018

Batch loss 490 : 0.003959294408559799

Batch loss 491 : 0.006850842386484146

Batch loss 492 : 0.01488605048507452

Batch loss 493 : 0.03676365688443184

Batch loss 494 : 0.0116676464676857

Batch loss 495 : 0.022349173203110695

Batch loss 496 : 0.004219611641019583

Batch loss 497 : 0.03632960468530655

Batch loss 498 : 0.02508026920258999

Batch loss 499 : 0.043021880090236664

Batch loss 500 : 0.005732561461627483

Batch loss 501 : 0.043354764580726624

Batch loss 502 : 0.00412213196977973

Batch loss 503 : 0.00475327717140317

Batch loss 504 : 0.0047193728387355804

Batch loss 505 : 0.021712999790906906

Batch loss 506 : 0.02310110256075859

Batch loss 507 : 0.002991974353790283

Batch loss 508 : 0.010022648610174656

Batch loss 509 : 0.04220671206712723

Batch loss 510 : 0.007779636420309544

Batch loss 511 : 0.004868764895945787

Batch loss 512 : 0.02580538205802


Batch loss 667 : 0.012398394756019115

Batch loss 668 : 0.010435440577566624

Batch loss 669 : 0.02401369996368885

Batch loss 670 : 0.0383731834590435

Batch loss 671 : 0.04592292383313179

Batch loss 672 : 0.01859920471906662

Batch loss 673 : 0.0030653325375169516

Batch loss 674 : 0.025405151769518852

Batch loss 675 : 0.02512255124747753

Batch loss 676 : 0.030344026163220406

Batch loss 677 : 0.03798877075314522

Batch loss 678 : 0.0030785587150603533

Batch loss 679 : 0.011666737496852875

Batch loss 680 : 0.0186435729265213

Batch loss 681 : 0.02460787259042263

Batch loss 682 : 0.03706829622387886

Batch loss 683 : 0.030917629599571228

Batch loss 684 : 0.04480523243546486

Batch loss 685 : 0.015188842080533504

Batch loss 686 : 0.01159312017261982

Batch loss 687 : 0.0016787860076874495

Batch loss 688 : 0.01489298976957798

Batch loss 689 : 0.018328411504626274

Batch loss 690 : 0.026712900027632713

Batch loss 691 : 0.027084365487098694

Batch loss 692 : 0.0194827765226364


Validation Batch loss 67 : 0.03046923689544201

Validation Batch loss 68 : 0.028673682361841202

Validation Batch loss 69 : 0.019565805792808533

Validation Batch loss 70 : 0.03403336554765701

Validation Batch loss 71 : 0.035757094621658325

Validation Batch loss 72 : 0.015925507992506027

Validation Batch loss 73 : 0.06021106615662575

Validation Batch loss 74 : 0.04014259949326515

Validation Batch loss 75 : 0.05877732113003731

Validation Batch loss 76 : 0.029679028317332268

Validation Batch loss 77 : 0.018179161474108696

Validation Batch loss 78 : 0.0364040769636631

Validation Batch loss 79 : 0.06498680263757706

Validation Batch loss 80 : 0.06645414978265762

Validation Batch loss 81 : 0.06210671737790108

Validation Batch loss 82 : 0.03697787597775459

Validation Batch loss 83 : 0.030623607337474823

Validation Batch loss 84 : 0.0270727276802063

Validation Batch loss 85 : 0.02430027909576893

Validation Batch loss 86 : 0.02296491712331772

Validation Batch loss 87 : 0.05153

OutOfMemoryError: CUDA out of memory. Tried to allocate 188.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 10.16 GiB is allocated by PyTorch, and 336.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [10]:
print(torch.cuda.memory_stats())

OrderedDict({'active.all.allocated': 2189543, 'active.all.current': 759, 'active.all.freed': 2188784, 'active.all.peak': 1024, 'active.large_pool.allocated': 1020003, 'active.large_pool.current': 277, 'active.large_pool.freed': 1019726, 'active.large_pool.peak': 381, 'active.small_pool.allocated': 1169540, 'active.small_pool.current': 482, 'active.small_pool.freed': 1169058, 'active.small_pool.peak': 721, 'active_bytes.all.allocated': 8415742742016, 'active_bytes.all.current': 10911057408, 'active_bytes.all.freed': 8404831684608, 'active_bytes.all.peak': 10917021184, 'active_bytes.large_pool.allocated': 8239346608128, 'active_bytes.large_pool.current': 10908502528, 'active_bytes.large_pool.freed': 8228438105600, 'active_bytes.large_pool.peak': 10913417728, 'active_bytes.small_pool.allocated': 176396133888, 'active_bytes.small_pool.current': 2554880, 'active_bytes.small_pool.freed': 176393579008, 'active_bytes.small_pool.peak': 13091328, 'allocated_bytes.all.allocated': 8415742742016, '

In [ ]:
print(inputs.input_ids.max())
print(inputs.input_ids.min())